In [ ]:
import os

current_path = os.getcwd()

project_root = os.path.abspath(os.path.join(current_path, '..', '..'))

In [ ]:
import numpy as np

SP = np.load(project_root+'/output/signal-planting-fo/SP.npy')
SA = np.load(project_root+'/output/signal-planting-fo/SA.npy')
SG = np.load(project_root+'/output/signal-planting-fo/SG.npy')
SE = np.load(project_root+'/output/signal-planting-fo/SE.npy')

lbsP = np.load(project_root+'/output/signal-planting-fo/lbsP.npy')
lbsA = np.load(project_root+'/output/signal-planting-fo/lbsA.npy')
lbsG = np.load(project_root+'/output/signal-planting-fo/lbsG.npy')
lbsE = np.load(project_root+'/output/signal-planting-fo/lbsE.npy')

n_valuesE = np.load(project_root+'/output/signal-planting-fo/n_valuesE.npy')
n_valuesG = np.load(project_root+'/output/signal-planting-fo/n_valuesG.npy')
n_valuesA = np.load(project_root+'/output/signal-planting-fo/n_valuesA.npy')
n_valuesP = np.load(project_root+'/output/signal-planting-fo/n_valuesP.npy')

n_valuesS = np.load(project_root+'/output/signal-planting-fo/n_valuesS.npy')

lbs = [lbsE,lbsG,lbsA,lbsP]
S = [SE,SG,SA,SP]
n_values = [n_valuesE,n_valuesG,n_valuesA,n_valuesP]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from tueplots import bundles, axes
from scipy.optimize import curve_fit
from tueplots import markers

%matplotlib widget

plt.rcParams.update(bundles.icml2022())
plt.rcParams.update(markers.inverted())
plt.rcParams.update(axes.grid())

colors = ['1f77b4','17becf'] 

scale_param = 10000 # used for fitting the sigmoid (adjust if needed)
x_valuesS = n_valuesS/scale_param
labels = ['Excellent', 'Good', 'Average', 'Poor']

def sigmoid(x, a, b, c, d): 
    return d + c/(1.0 + np.exp(-a*(x-b)))

# bounds for sigmoid fitting (adjust if needed)
amin = [0,-1,0,0]
amax = [0,1,0,0]
bmin = [0,-1,-1,-1]
bmax = [0,1,0,0]
cmin = [-5,-1,-3,-3]
cmax = [10,3,5,5]
dmin = [-0.5,-0.5,-1,-1]
dmax = [0.5,1,0,0]
aminb = [-5,0,0,0]
amaxb = [5,0,0,0]
bminb = [-5,0,0,0]
bmaxb = [5,0,0,0]
cminb = [-10,-1,-1,-1]
cmaxb = [10,10,10,10]
dminb = [-1,-0.5,-1,-1]
dmaxb = [1,1,0,0]

fig, axes = plt.subplots(1, 4, figsize=(9, 2)) 

for i, ax in enumerate(axes):
    x_values = n_values[i]/scale_param

    y_values = lbs[i]
    y_valuesb = S[i]

    # sigmoid lower bound
    popt, pcov = curve_fit(sigmoid, x_values, y_values, method='dogbox', bounds=([min(y_values)+amin[i],min(x_values)+bmin[i],cmin[i],dmin[i]],[max(y_values)+amax[i],max(x_values)+bmax[i],cmax[i],dmax[i]]))
    x_fit = np.linspace(min(x_values), max(x_values))
    y_fit = np.minimum(sigmoid(x_fit, *popt),1)
    ax.plot(x_fit, y_fit, label='Lower bound',linewidth=3.5,zorder=1,color='#'+colors[0])

    # sigmoid success
    popt, pcov = curve_fit(sigmoid, x_valuesS, y_valuesb, method='dogbox', bounds=([min(y_valuesb)+aminb[i],min(x_values)+bminb[i],cminb[i],dminb[i]],[max(y_valuesb)+amaxb[i],max(x_values)+bmaxb[i],cmaxb[i],dmaxb[i]]))
    x_fit = np.linspace(min(x_values), max(x_values))
    y_fit = np.minimum(sigmoid(x_fit, *popt),1)
    ax.plot(x_fit, y_fit, label=r'$\hat{S}(n)$',linewidth=3.5, linestyle=(0,(1,1)),zorder=1,color='#'+colors[1])

    # points success
    ax.scatter(x_valuesS,y_valuesb,marker='o',edgecolor='#'+colors[1],color='white',linewidth=1,zorder=2,s=50)
    # points lower bound
    ax.scatter(x_values,y_values,marker='o',edgecolor='#'+colors[0],color='white',linewidth=1,zorder=2,s=50)
    
    # add title and adjust tick labels
    ax.set_title(rf'$y^*$ = {labels[i]}', fontsize=12)
    ax.tick_params(axis='both', labelsize=10)
    ax.grid()
    ax.set_ylim(-0.05,1.05)
    # set x-ticks
    ax.set_xticks(np.arange(0, max(x_values) + 1, 5), minor=True) 
    ax.set_xticks(np.arange(0, max(x_values) + 1, 20), minor=False) 
    ax.set_xticklabels(np.arange(0, max(x_values) + 1, 20), minor=False)
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: int(x))) 
    ax.tick_params(axis='x', which='major', length=7)
    ax.tick_params(axis='x', which='minor', length=4) 
    ax.tick_params(axis='x', which='both', width=1) 
    # set y-ticks
    ax.set_yticks(np.arange(0, 101, 5)/100, minor=True) 
    ax.set_yticks(np.arange(0, 101, 20)/100, minor=False) 
    ax.set_yticklabels(np.arange(0, 101, 20)/100, minor=False)
    ax.tick_params(axis='y', which='major', length=7)
    ax.tick_params(axis='y', which='minor', length=4) 
    ax.tick_params(axis='y', which='both', width=1) 

# set shared labels and add a single legend
fig.supylabel('Success', fontsize=12, x=0.01, y=0.56)
fig.supxlabel(r'Relative collective size $n/N$ (in $\%$)', fontsize=12)
axes[0].legend(loc='lower right', fontsize=9)

plt.tight_layout()
plt.subplots_adjust(bottom=0.3, wspace=0.3) 

plt.savefig(project_root+"/plots/signal-planting-fo.pdf", format="pdf", bbox_inches="tight")

plt.show()